In [2]:
import pandas as pd
df = pd.read_csv(r'dataset\job_description.csv',encoding='ISO-8859-1')
df = df.iloc[:,:-1]


In [3]:
df.head()

,Job Title,Job Description
0,Software Engineer,Description:\nWe are seeking a skilled Softwa...
1,Data Scientist,Job Description:\nWe are looking for a skilled...
2,Product Manager,Description:\nWe are seeking an innovative and...
3,Cloud Engineer,Description:\nWe are looking for a skilled Clo...
4,Cybersecurity Analyst,Description:\nWe are looking for a skilled Cyb...


In [4]:
df.shape

(20, 2)

In [5]:
# Clean and combine title + description
combined_jds = df.apply(
    lambda row: f"Job Title: {row['Job Title']}\n\n{row['Job Description']}",
    axis=1
)

# Example: looping through each to pass to the LLM
for idx, jd_text in enumerate(combined_jds):
    print(f"--- JD #{idx+1} ---")
    print(jd_text)
    # Call your LLM chain here, e.g. summarizer_chain.run(job_description=jd_text)

--- JD #1 ---
Job Title: Software Engineer

 Description:
We are seeking a skilled Software Engineer to design, develop, and maintain software applications. The ideal candidate will write efficient code, troubleshoot issues, and collaborate with teams to deliver high-quality solutions.

Responsibilities:

Develop, test, and deploy software applications.
Write clean, maintainable, and scalable code.
Collaborate with cross-functional teams to define and implement features.
Troubleshoot and debug issues for optimal performance.
Stay updated with emerging technologies and best practices.
Qualifications:

Bachelor's degree in Computer Science or a related field.
Proficiency in programming languages like Python, Java, or C++.
Experience with databases, web development, and software frameworks.
Strong problem-solving skills and attention to detail.
Ability to work both independently and in a team environment.
--- JD #2 ---
Job Title: Data Scientist

Job Description:
We are looking for a skill

In [112]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.embeddings import OllamaEmbeddings
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import json

In [57]:
# i want to read JD desc one by one and struc them in suitable format.
llm = OllamaLLM (model = 'mistral:latest',temperature=1)

In [ ]:
template = """
   You will be given a job description. Your task is to extract ONLY the details mentioned in the text.
JD_text:{JD_text}
Extract and format the output as follows:
1. Job Role / Title  
2. Required Technical Skills (only if explicitly mentioned)  
3. Minimum Years of Experience Required (only if explicitly stated)  
4. Educational Qualifications  
5. Job Responsibilities (summarized using original words)  
6. Preferred Domain Expertise (only if mentioned)  
7. Other Key Requirements or Notes (include only what's in the text)
8. Stages of further candidate selection process (only if mentioned) 

Only use the information from the given job description. Do NOT add any assumptions or generalizations. Return the result in the specified bullet format. Do not return JSON or prose.


"""


prompt_temp = PromptTemplate.from_template(template)
print(prompt_temp)

input_variables=['JD_text'] input_types={} partial_variables={} template="\n   You will be given a job description. Your task is to extract ONLY the details mentioned in the text.\nJD_text:{JD_text}\nExtract and format the output as follows:\n1. Job Role / Title  \n2. Required Technical Skills (only if explicitly mentioned)  \n3. Minimum Years of Experience Required (only if explicitly stated)  \n4. Educational Qualifications  \n5. Job Responsibilities (summarized using original words)  \n6. Preferred Domain Expertise (only if mentioned)  \n7. Other Key Requirements or Notes (include only what's in the text)\n\nOnly use the information from the given job description. Do NOT add any assumptions or generalizations. Return the result in the specified bullet format. Do not return JSON or prose.\n\n\n"


In [69]:
chain = prompt_temp | llm
content = chain.invoke({"JD_text":combined_jds[1]})


In [70]:
print(content)

1. Job Role / Title: Data Scientist
  2. Required Technical Skills (only if explicitly mentioned): Proficiency in Python, R, SQL, and machine learning frameworks; Experience with data visualization tools like Tableau or Power BI
  3. Minimum Years of Experience Required (only if explicitly stated): Not specified in the text
  4. Educational Qualifications: Bachelor's or Master's degree in Data Science, Computer Science, or a related field.
  5. Job Responsibilities (summarized using original words): Collect, clean, and analyze large datasets; Develop and deploy machine learning models; Build predictive analytics solutions to improve business outcomes; Communicate findings through reports and visualizations; Stay updated with advancements in data science and AI.
  6. Preferred Domain Expertise (only if mentioned): Not mentioned in the text
  7. Other Key Requirements or Notes (include only what's in the text): Ability to work independently and in a team environment.


In [137]:
def convert_to_json(text:str):
    try:
        json_data = json.loads(text)
        print(json_data)
        print(type(json_data))
        return json_data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

In [71]:
formatted_JDs = []
for idx,jd in enumerate(combined_jds):
    print(f"--- JD {idx+1} ----")
    content = chain.invoke({"JD_text":jd})
    formatted_JDs.append(content)


--- JD 1 ----
--- JD 2 ----
--- JD 3 ----
--- JD 4 ----
--- JD 5 ----
--- JD 6 ----
--- JD 7 ----
--- JD 8 ----
--- JD 9 ----
--- JD 10 ----
--- JD 11 ----
--- JD 12 ----
--- JD 13 ----
--- JD 14 ----
--- JD 15 ----
--- JD 16 ----
--- JD 17 ----
--- JD 18 ----
--- JD 19 ----
--- JD 20 ----


In [75]:
print(formatted_JDs[7])

1. Job Role / Title: Full Stack Developer
  2. Required Technical Skills: JavaScript, React, Node.js, databases (SQL/NoSQL), RESTful APIs, cloud services, version control (Git)
  3. Minimum Years of Experience Required: Not explicitly stated
  4. Educational Qualifications: Bachelor's degree in Computer Science or related field
  5. Job Responsibilities: Develop and maintain web applications, collaborate with designers and backend engineers, optimize application performance and ensure scalability, troubleshoot and debug issues in a fast-paced environment, stay updated with emerging web technologies and best practices
  6. Preferred Domain Expertise: Not explicitly mentioned
  7. Other Key Requirements or Notes: Strong problem-solving skills and ability to work in a collaborative team


In [73]:
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")


In [76]:
df["Job Title"][1]

'Data Scientist'

In [80]:
query_embedding = {
    i: {
        "job_title": df["Job Title"][i],
        "embedding": embedding_model.embed_query(formatted_JDs[i])
    }
    for i in range(len(df))
}


In [82]:
query_embedding[3]

{'job_title': 'Cloud Engineer',
 'embedding': [-0.8775838017463684,
  0.7852258682250977,
  -2.8432974815368652,
  -0.3997351825237274,
  0.9002860188484192,
  0.6405858993530273,
  0.6475040912628174,
  -0.18744170665740967,
  -0.08355152606964111,
  -0.9659029841423035,
  -0.3797895908355713,
  -0.1047317385673523,
  1.3096051216125488,
  0.35068991780281067,
  0.18466007709503174,
  0.5301744937896729,
  -0.4440518915653229,
  -1.0275675058364868,
  0.10731888562440872,
  -0.03143341839313507,
  -0.5473591685295105,
  -2.3626129627227783,
  0.4971984624862671,
  -0.3601486384868622,
  0.5352105498313904,
  0.6835395097732544,
  -0.20584988594055176,
  -0.4541177749633789,
  -0.605570375919342,
  -0.7029808163642883,
  0.6322735548019409,
  0.33341994881629944,
  -0.5232754349708557,
  -1.071351408958435,
  -1.268657922744751,
  -1.2588129043579102,
  0.9610434770584106,
  0.32807987928390503,
  0.07716196030378342,
  0.3248133659362793,
  0.8351925015449524,
  -0.7673128843307495,
 

In [ ]:
# CREATE TABLE IF NOT EXISTS User (
#     userid INTEGER PRIMARY KEY AUTOINCREMENT,
#     username varchar(50) NOT NULL,
#     email varchar(50) UNIQUE NOT NULL,
#     password varchar(50) NOT NULL,
#     role TEXT CHECK(role IN ('admin', 'user')) NOT NULL,
# 	phone varchar(10)
# );
# CREATE TABLE IF NOT EXISTS Job (
#     jobid INTEGER PRIMARY KEY AUTOINCREMENT,
#     company varchar(50) NOT NULL,
#     role varchar(50) NOT NULL,
#     description TEXT NOT NULL,
#     skills varchar(50) NOT NULL,
#     openings INTEGER NOT NULL,
#     salary INTEGER,
#     created_by INTEGER,
#     FOREIGN KEY (created_by) REFERENCES User(userid)
# );
# CREATE TABLE IF NOT EXISTS Application (
#     applicationid INTEGER PRIMARY KEY AUTOINCREMENT,
#     userid INTEGER NOT NULL,
#     jobid INTEGER NOT NULL,
#     resume TEXT,  
# 	status TEXT DEFAULT 'Pending' CHECK(status IN ('Pending', 'Accepted', 'Rejected')),
#     applied_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#     FOREIGN KEY (userid) REFERENCES User(userid),
#     FOREIGN KEY (jobid) REFERENCES Job(jobid),
#     UNIQUE(userid, jobid)  -- Prevent duplicate applications
# );


In [ ]:
import sqlite3
# Connect to (or create) a database file
conn = sqlite3.connect(r"db\CVAgentDB.db")
cursor = conn.cursor()

# Create a table
cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS User (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username VARCHAR(50) NOT NULL,
    password VARCHAR(50) NOT NULL,
    CV TEXT,
    dateofinterview DATE,
    role TEXT CHECK( role IN ('user','admin'))   NOT NULL DEFAULT 'admin',
    email VARCHAR(50) NOT NULL,
    phone VARCHAR(10),
    jobid INTEGER,
    FOREIGN KEY(jobid) REFERENCES Job(jobid)
);

''')
cursor.execute(f'''
  CREATE TABLE Job (
    jobid INTEGER PRIMARY KEY AUTOINCREMENT,
    company TEXT NOT NULL,
    role TEXT,
    JD TEXT,
    skills TEXT,
    openings INTEGER,
    salary TEXT,
    created_by INTEGER,
    FOREIGN KEY(created_by) REFERENCES User(id)
);


''')


conn.commit()
conn.close()

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect(r"db\CVAgentDB.db")
cursor = conn.cursor()

# Sample job data
job_data = [
    ("Software Engineer","""Job Title: Software Engineer

 Description:
We are seeking a skilled Software Engineer to design, develop, and maintain software applications. The ideal candidate will write efficient code, troubleshoot issues, and collaborate with teams to deliver high-quality solutions.

Responsibilities:

Develop, test, and deploy software applications.
Write clean, maintainable, and scalable code.
Collaborate with cross-functional teams to define and implement features.
Troubleshoot and debug issues for optimal performance.
Stay updated with emerging technologies and best practices.
Qualifications:

Bachelor's degree in Computer Science or a related field.
Proficiency in programming languages like Python, Java, or C++.
Experience with databases, web development, and software frameworks.
Strong problem-solving skills and attention to detail.
Ability to work both independently and in a team environment.""", "Python, Flask, REST, SQL", 3, "12-18 LPA"),
    ("Frontend Developer", "ReactJS developer needed to build responsive UIs for dashboards and web apps.", "JavaScript, React, HTML, CSS", 2, "10-15 LPA"),
    ("Data Analyst", "Analyze large datasets, build dashboards, and provide insights for business decisions.", "SQL, Power BI, Excel, Python", 2, "8-12 LPA"),
    ("DevOps Engineer", "Responsible for CI/CD pipelines, Docker, Kubernetes, and cloud deployment.", "Docker, Kubernetes, Jenkins, AWS", 1, "15-20 LPA"),
]

# Insert sample jobs
for job in job_data:
    cursor.execute("""
        INSERT INTO Job (role, JD, skills, openings, salary)
        VALUES (?, ?, ?, ?, ?)
    """, job)

# Commit and close
conn.commit()
conn.close()

print("Sample job postings inserted successfully.")


Sample job postings inserted successfully.


In [107]:
def readPDF(path:str)->str:
   # importing required classes
    from pypdf import PdfReader

    # creating a pdf reader object
    reader = PdfReader(path)

    # printing number of pages in pdf file
    print(len(reader.pages))
    pdf_text = ""
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        pdf_text = pdf_text + page.extract_text()
        print(page.extract_text())
    
    return pdf_text

In [98]:
import os
from pathlib import Path
directory = "dataset\CVs1"
for filename in os.listdir(directory):
    readPDF(str(Path(directory+'/'+filename)))


<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\harsh\AppData\Local\Temp\ipykernel_48972\3475033291.py:3: SyntaxWarning: invalid escape sequence '\C'
  directory = "dataset\CVs1"


1
Candidate Resume (ID: C1061)
Name: Alyssa Chavez
Email: alyssachavez88@gmail.com
Phone: +1-465-3587
Education
Diploma in Software Engineering (2013-2015)
Hands-on experience in full-stack web development and mobile app creation.
Work Experience
Data Scientist at ABC Inc. (2019-2023)
Built predictive models that enhanced decision-making processes, reducing operational costs by
25%.
Skills
Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks
against cyber threats.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry
leaders, shaping responsible AI development.
Tech Stack
Java, Spring Boot, MySQL, Kafka, Azure DevOps
2
Candidate Resume (ID: C1070)
Name: Scott Saunders
Email: scottsaunders13@gmail.com
Phone: +1-367-5130
Educatio

LLM 2 CV parser agent

In [99]:
llm2 = OllamaLLM(model= "mistral", temperature=1)

In [103]:
CVtemplate = """
   You are an AI assistant designed to extract key structured information from a candidate's resume. 
    CV_text:{CV_text}
Your goal is to accurately and concisely extract the following elements:

1. **Technical Skills**: Provide a bullet point list of technical skills mentioned (e.g., programming languages, tools, frameworks).
2. **Years of Experience**: Provide a single numeric value (approximate if not clearly mentioned).
3. **Education Level**: Mention the highest level of education attained (e.g., Bachelor's in Computer Science).
4. **Experience Level**: Categorize into one of the following: Junior / Mid-level / Senior based on role and total experience.
5. **Key Achievements**: Summarize 2–3 impactful accomplishments or responsibilities (avoid generic job duties).
6. **Domain Expertise**: Specify any domains/industries the candidate has worked in (e.g., FinTech,
Only use the information from the given resume. Do NOT add any assumptions or generalizations. Return the result in the specified bullet format. Do not return JSON or prose.

"""


prompt_CV = PromptTemplate.from_template(CVtemplate)
print(prompt_CV)


input_variables=['CV_text'] input_types={} partial_variables={} template="\n   You are an AI assistant designed to extract key structured information from a candidate's resume. \n    CV_text:{CV_text}\nYour goal is to accurately and concisely extract the following elements:\n\n1. **Technical Skills**: Provide a bullet point list of technical skills mentioned (e.g., programming languages, tools, frameworks).\n2. **Years of Experience**: Provide a single numeric value (approximate if not clearly mentioned).\n3. **Education Level**: Mention the highest level of education attained (e.g., Bachelor's in Computer Science).\n4. **Experience Level**: Categorize into one of the following: Junior / Mid-level / Senior based on role and total experience.\n5. **Key Achievements**: Summarize 2–3 impactful accomplishments or responsibilities (avoid generic job duties).\n6. **Domain Expertise**: Specify any domains/industries the candidate has worked in (e.g., FinTech,\nOnly use the information from th

In [108]:
chain = prompt_CV | llm2
content = chain.invoke({"CV_text":readPDF(r'dataset\CVs1\C1070.pdf')})


2
Candidate Resume (ID: C1070)
Name: Scott Saunders
Email: scottsaunders13@gmail.com
Phone: +1-367-5130
Education
Bachelor of Engineering in Information Technology (2014-2018)
Concentrated on database management, networking, and cybersecurity.
Master of Business Administration (2017-2019)
Focused on Business Strategy, Financial Analysis, and Operations Management.
Ph.D. in Artificial Intelligence (2016-2021)
Research in NLP and Computer Vision, with publications in top-tier conferences.
Work Experience
Software Engineer at XYZ Corp (2018-2022)
Developed scalable backend applications, improved system efficiency by 30%, and led agile
development sprints.
Skills
Python & Machine Learning - Proficient in TensorFlow, PyTorch, and Scikit-learn with hands-on
experience in deploying AI solutions.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Developed an AI chatbot - Bu

In [109]:
print(content)

1. Technical Skills:
     - Python
     - Machine Learning (TensorFlow, PyTorch, Scikit-learn)
     - AWS Solutions Architect
     - NLP (Natural Language Processing)
     - Computer Vision
     - PostgreSQL
     - Docker
     - Kubernetes

  2. Years of Experience: 2022 - 2018 = 4 years (excluding Ph.D.)

  3. Education Level: Ph.D. in Artificial Intelligence

  4. Experience Level: Senior (based on a combination of higher education level and significant work experience)

  5. Key Achievements:
     - Developed an AI chatbot that reduced customer support tickets by 40%.
     - Improved system efficiency by 30% at XYZ Corp.
     - Led agile development sprints at XYZ Corp.

  6. Domain Expertise: N/A (No specific industry experience was mentioned in the given resume)


llm3 matcher agent

In [119]:
matcher_template = """
You are a recruiting assistant tasked with evaluating how well a candidate matches a job description.
You will be provided with two structured documents:
A Candidate CV Summary
A Job Description Summary

CV = {CV}
JD = {JD}
Based on the information, do the following:
Calculate a Match Score out of 100, giving higher weight to:
Skill overlap (40%)
Relevant years of experience (20%)
Education level (10%)
Domain expertise (20%)
Certifications & achievements (10%)
Provide a short explanation (2–3 sentences) of how this score was derived.
Make a recommendation:
Proceed to Interview if the score is 80 or above
Reject if the score is below 80

Respond in the following JSON format:
{{
"Match_Score": <score>/100
"Recommendation": <Proceed to Interview / Reject>
"Reason": <concise explanation>
}}

"""

matcher_prompt = PromptTemplate.from_template(matcher_template)
print(matcher_prompt)

input_variables=['CV', 'JD'] input_types={} partial_variables={} template='\nYou are a recruiting assistant tasked with evaluating how well a candidate matches a job description.\nYou will be provided with two structured documents:\nA Candidate CV Summary\nA Job Description Summary\n\nCV = {CV}\nJD = {JD}\nBased on the information, do the following:\nCalculate a Match Score out of 100, giving higher weight to:\nSkill overlap (40%)\nRelevant years of experience (20%)\nEducation level (10%)\nDomain expertise (20%)\nCertifications & achievements (10%)\nProvide a short explanation (2–3 sentences) of how this score was derived.\nMake a recommendation:\nProceed to Interview if the score is 80 or above\nReject if the score is below 80\n\nRespond in the following JSON format:\n{{\n"Match_Score": <score>/100\n"Recommendation": <Proceed to Interview / Reject>\n"Reason": <concise explanation>\n}}\n\n'


In [120]:
llm3 = OllamaLLM(model='mistral',temperature=0.5)

In [123]:
from langchain_core.runnables import RunnableMap

chain = RunnableMap({
    "CV": lambda x: readPDF(x["path"]),
    "JD": lambda x: formatted_JDs[0]
}) | matcher_prompt | llm3

In [124]:
content = chain.invoke({"path":r"dataset\CVs1\C1061.pdf"})

1
Candidate Resume (ID: C1061)
Name: Alyssa Chavez
Email: alyssachavez88@gmail.com
Phone: +1-465-3587
Education
Diploma in Software Engineering (2013-2015)
Hands-on experience in full-stack web development and mobile app creation.
Work Experience
Data Scientist at ABC Inc. (2019-2023)
Built predictive models that enhanced decision-making processes, reducing operational costs by
25%.
Skills
Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks
against cyber threats.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry
leaders, shaping responsible AI development.
Tech Stack
Java, Spring Boot, MySQL, Kafka, Azure DevOps


In [138]:
json_data = convert_to_json(content)

{'Match_Score': 78, 'Recommendation': 'Proceed to Interview', 'Reason': 'While Alyssa Chavez has a strong background in software engineering with relevant skills like cybersecurity and proficiency in Java, her education level is only at the diploma level (10%). Additionally, she lacks direct experience with preferred languages like Python or C++ (20%), but her expertise in other areas such as AWS and Kafka compensates for this to some extent. Her domain expertise in data science may not be a perfect fit for a Software Engineer role, but her achievements in AI ethics demonstrate problem-solving skills and attention to detail (10%). Lastly, the score is boosted by her relevant work experience at ABC Inc., although no specific years are provided (20%).'}
<class 'dict'>


llm4 email generator agent

In [133]:
llm4 = OllamaLLM(model="mistral",temperature=0.9)

In [153]:
email_template = """
You are an AI assistant that drafts professional emails for recruitment decisions.

You will be provided with:
- Match Score
- Recommendation (Proceed to Interview or Reject)
- Reason (evaluation reasoning from the matcher agent)

Your job is to:
1. Write a professional email based on the recommendation.
2. If the recommendation is "Proceed to Interview", congratulate the candidate, notify them regarding further stages of recruitment (only if available) and let them know they will be contacted to schedule an interview. DO NOT MENTION REASON for selection.
3. If the recommendation is "Reject", politely inform the candidate, summarize and mention the reason and appreciate their interest.
4. Keep it formal, polite, and concise.
5. The email must end with a signature from the sender as **'CVAgent Team'**.

Output response must be in the following JSON format:
{{
    "subject":"<subject_line>",
    "body":"<email_body>"
}}

Here is the input:
Match Score: {score}/100  
Recommendation: {status}  
Reason: {reason}
"""

email_prompt = PromptTemplate.from_template(email_template)
print(email_prompt)


input_variables=['reason', 'score', 'status'] input_types={} partial_variables={} template='\nYou are an AI assistant that drafts professional emails for recruitment decisions.\n\nYou will be provided with:\n- Match Score\n- Recommendation (Proceed to Interview or Reject)\n- Reason (evaluation reasoning from the matcher agent)\n\nYour job is to:\n1. Write a professional email based on the recommendation.\n2. If the recommendation is "Proceed to Interview", congratulate the candidate, notify them regarding further stages of recruitment (only if available) and let them know they will be contacted to schedule an interview. DO NOT MENTION REASON for selection.\n3. If the recommendation is "Reject", politely inform the candidate, summarize and mention the reason and appreciate their interest.\n4. Keep it formal, polite, and concise.\n5. The email must end with a signature from the sender as **\'CVAgent Team\'**.\n\nOutput response must be in the following JSON format:\n{{\n    "subject":"<s

In [154]:
chain = email_prompt | llm4

In [155]:
resp = chain.invoke({"score": json_data["Match_Score"] , "status": json_data['Recommendation'] , "reason": json_data['Reason']})

In [ ]:
print(resp)
resp = convert_to_json(resp)

NameError: name 'resp' is not defined

In [165]:
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
print(os.getenv('EMAIL_APP_PASSWORD'))


cvap uoxy wpdr osed


In [1]:
 # Generate email
from agents.generator import GeneratorAgent
from email_service import EmailService
# generator = GeneratorAgent()

# email_str = generator.run({"score": json_data["Match_Score"] , "status": json_data['Recommendation'] , "reason": json_data['Reason']})
# email_json = convert_to_json(email_str)

# Send email
eservice = EmailService("devarshan1853@gmail.com")
isSent = eservice.send_email("arshan5446@gmail.com",resp["subject"], resp["body"])


NameError: name 'resp' is not defined